In [12]:
i0=1
i1=2
i2=-1

In [13]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
from tqdm import tqdm
import sys
sys.path.append('/home/icb/manuel.gander/Atl/notebooks/')
import utils
utils = importlib.reload(utils)
import scipy
import pickle

import warnings
from scipy.stats import ConstantInputWarning
from sklearn.exceptions import ConvergenceWarning

# Ignore ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
kns = [10, 30, 100, 300, 1000, 3000, 10000]
reps=100

In [14]:
alpha = alphas[i0]
l1_ratio = l1_ratios[i1]
kn = kns[i2]

source = 'GDSC2'
dataset = 'atl_only_phos'

In [31]:
D_prot, features, celllines = utils.load_dataset(dataset)
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']==source].copy()
dfv = dfv[np.isfinite(dfv['AUC'])].copy()    

with open('/home/icb/manuel.gander/Atl/data/common_drugs.pkl', 'rb') as file:
    Dd = pickle.load(file)
drugs = Dd[source]


frames=[]
for dr in tqdm(drugs[:1]):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    res['td']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps, source0='all')
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                t0=time.time()
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                td = time.time()-t0
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))
                res['td'].append(td)

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['source']=source
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['kn'] = kn

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.17s/it]


In [30]:
df

,pearsons,spearmans,l1_ratios,RMSE,RMSE_mean_model,td,PubChem_CID,ind,n_ccls
0,0.747013,0.785714,0.604829,0.132844,0.197262,0.297047,11707110,0,82
1,0.746847,0.671429,0.619585,0.157976,0.224345,0.297047,11707110,1,82
2,0.705401,0.727473,0.635490,0.134707,0.191689,0.297047,11707110,2,82
3,0.609775,0.525000,0.718063,0.137288,0.168633,0.297047,11707110,3,82
4,0.404213,0.377622,0.838647,0.167199,0.172158,0.297047,11707110,4,82
...,...,...,...,...,...,...,...,...,...
95,0.490209,0.476923,0.812731,0.192582,0.215271,0.297047,11707110,95,82
96,0.543993,0.433444,0.690360,0.187132,0.218689,0.297047,11707110,96,82
97,0.786861,0.754676,0.592377,0.109351,0.170556,0.297047,11707110,97,82
98,0.712159,0.646865,0.760260,0.140336,0.194061,0.297047,11707110,98,82


In [28]:
t0=time.time()
predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
td = time.time()-t0

In [29]:
td

0.22980642318725586

In [17]:
Df

,pearsons,spearmans,l1_ratios,RMSE,RMSE_mean_model,td,PubChem_CID,ind,n_ccls,source,dataset,alpha,l1_ratio,kn
0,0.747013,0.785714,0.604829,0.132844,0.197262,0.297047,11707110,0,82,GDSC2,atl_only_phos,0.03,0.1,10000
1,0.746847,0.671429,0.619585,0.157976,0.224345,0.297047,11707110,1,82,GDSC2,atl_only_phos,0.03,0.1,10000
2,0.705401,0.727473,0.635490,0.134707,0.191689,0.297047,11707110,2,82,GDSC2,atl_only_phos,0.03,0.1,10000
3,0.609775,0.525000,0.718063,0.137288,0.168633,0.297047,11707110,3,82,GDSC2,atl_only_phos,0.03,0.1,10000
4,0.404213,0.377622,0.838647,0.167199,0.172158,0.297047,11707110,4,82,GDSC2,atl_only_phos,0.03,0.1,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.490209,0.476923,0.812731,0.192582,0.215271,0.297047,11707110,95,82,GDSC2,atl_only_phos,0.03,0.1,10000
96,0.543993,0.433444,0.690360,0.187132,0.218689,0.297047,11707110,96,82,GDSC2,atl_only_phos,0.03,0.1,10000
97,0.786861,0.754676,0.592377,0.109351,0.170556,0.297047,11707110,97,82,GDSC2,atl_only_phos,0.03,0.1,10000
98,0.712159,0.646865,0.760260,0.140336,0.194061,0.297047,11707110,98,82,GDSC2,atl_only_phos,0.03,0.1,10000


In [16]:
Df

,pearsons,spearmans,l1_ratios,RMSE,RMSE_mean_model,td,PubChem_CID,ind,n_ccls,source,dataset,alpha,l1_ratio,kn
0,0.747013,0.785714,0.604829,0.132844,0.197262,0.297047,11707110,0,82,GDSC2,atl_only_phos,0.03,0.1,10000
1,0.746847,0.671429,0.619585,0.157976,0.224345,0.297047,11707110,1,82,GDSC2,atl_only_phos,0.03,0.1,10000
2,0.705401,0.727473,0.635490,0.134707,0.191689,0.297047,11707110,2,82,GDSC2,atl_only_phos,0.03,0.1,10000
3,0.609775,0.525000,0.718063,0.137288,0.168633,0.297047,11707110,3,82,GDSC2,atl_only_phos,0.03,0.1,10000
4,0.404213,0.377622,0.838647,0.167199,0.172158,0.297047,11707110,4,82,GDSC2,atl_only_phos,0.03,0.1,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.490209,0.476923,0.812731,0.192582,0.215271,0.297047,11707110,95,82,GDSC2,atl_only_phos,0.03,0.1,10000
96,0.543993,0.433444,0.690360,0.187132,0.218689,0.297047,11707110,96,82,GDSC2,atl_only_phos,0.03,0.1,10000
97,0.786861,0.754676,0.592377,0.109351,0.170556,0.297047,11707110,97,82,GDSC2,atl_only_phos,0.03,0.1,10000
98,0.712159,0.646865,0.760260,0.140336,0.194061,0.297047,11707110,98,82,GDSC2,atl_only_phos,0.03,0.1,10000
